## Построение тематической модели классификации коллекции EUR-lex.

### Мурат Апишев, great-mel@yandex.ru

#### Описание эксперимента и датасета

В данном эксперименте мы будем строить тематическую модель коллекции EUR-lex, в которой имеются две модальности --- слова и метки классов. Она имеет следующие характеристики (после предобработки):
- 20000 документов, около 18000 разбитых в батчи обучающей выборки по 1000 штук, и примерно 1950 лежащих в отдельном тестовом батче;
- 21000 слов в словаре после предобработки;
- 3900 меток классов;
- каждый документ относится в среднем к 3-6 классам.

Цель эксперимента --- построить качественную тематическую модель классификации. Критериями качества являются:
- площадь под ROC-кривой (AUC-ROC)
- площадь под кривой precision-recall (AUC-PR)
- доля документов, у которых самая вероятная метка класса оказалась неверной (OneError)
- доля документов, которые не были классифицированы идеально (IsError)
- средняя точность: для каждой верной метки считается доля верных меток, ранжированных выше, после чего происходит усреднение внутри документа и по всем документам (AverPrec). 

AUC-характеристики считаются между вектором вероятностей классов и вектором верных ответов для одного документа, после чего происходит усреднение по всем документам.

Все описанные величины считаются на тестовой выбоке, документы тестового батча не содержат информации о своих метках классов.

Все метрики взяты из статьи T. Rubin, A. Chambers, P. Smyth, M. Steyvers: Statistical topic models for multi-label document classification.

#### Описание хода эксперимента

Прежде всего подключим необходимые пакеты Python и новый API BigARTM:

In [1]:
from __future__ import division

import os
import sys
import glob
import pickle
import numpy
import sklearn.metrics

import artm

Далее определим две вспомогательные функции, которые будут использоваться при вычислении функционалов качества:

In [2]:
def perfect_classification(true_labels, probs):
    temp_true_labels = list(true_labels)
    temp_probs = list(probs)
    for i in xrange(sum(true_labels)):
        idx = temp_probs.index(max(temp_probs))
        
        if temp_true_labels[idx] == 0:
            return False
        
        del temp_true_labels[idx]
        del temp_probs[idx]
    return True

In [3]:
def count_precision(true_labels, probs):
    retval, index = 0, -1
    for label in true_labels:
        denominator, numerator = 0, 0
        index += 1
        if label:
            for prob_idx in xrange(len(probs)):
                if probs[prob_idx] > probs[index]:
                    denominator += 1
                    if true_labels[prob_idx] == 1:
                        numerator += 1
        if denominator > 0:
            retval += numerator / denominator
    retval /= sum(true_labels)
    return retval

Теперь определим несколько полезных статических констант. Это
- имена модальностей (те, что использовались парсером при создании батчей и словаря);
- полное имя директории с батчами;
- полное имя файла с информацией о метках тестовых документов;
- имя файла с расширением '.batch_test', содержащего тестовые документы.

In [4]:
labels_class = '@labels_class'
tokens_class = '@default_class'

data_folder         = 'D:/Work/University/course_work/bigartm/multimodal_experiments/eurlex_data'
test_labels_file    = os.path.join(data_folder, 'test_labels.eurlex_artm')
test_documents_file = '7d6a65e7-712a-43e5-bdad-529075961598.batch_test'

Сразу загрузим из файла информацию о тестовых метках:

In [5]:
with open(test_labels_file, 'rb') as f:
    true_p_cd = [[int(p_cd) for p_cd in p_d] for p_d in pickle.load(f)]

Наша модель будет характеризоваться определённым набором параметров, а именно
- числом тем
- числом итераций по коллекции
- числом итераций по документу (+)
- весом модальности "метки классов" (+)
- весом модальности "слова" (+)
- коэффициентом сглаживания Теты (+)
- коэффициентом сглаживания Фи (+)
- коэффициентом сглаживания Пси (+)
- коэффициентом регуляризатора балансирования классов (+)

(+) --- величина представляет собой список значений, по одному на каждую итерацию прохода по коллекции.

Кроме этих основных величин надо задать технический список номеров итераций, на которых следует подсчитывать значения метрик.

In [6]:
num_topics            = 100
num_collection_passes = 5

num_document_passes   = [16] * num_collection_passes
labels_class_weight   = [1.0, 1.0, 0.9, 0.9, 0.9, 0.8, 0.8, 0.8, 0.7, 0.7]
tokens_class_weight   = [1] * num_collection_passes

smooth_theta_tau      = [0.02] * num_collection_passes
smooth_phi_tau        = [0.01] * num_collection_passes

smooth_psi_tau        = [0.01] * num_collection_passes
label_psi_tau         = [0.0] * num_collection_passes

count_scores_iters = [num_collection_passes - 1]

Теперь, собственно, создадим модель, после чего инициализируем её с помощью словаря коллекции:

In [7]:
model = artm.ArtmModel(num_topics=num_topics, num_document_passes=1)

In [8]:
model.load_dictionary(dictionary_name='dictionary', dictionary_path=os.path.join(data_folder, 'dictionary.eurlex_artm'))
model.initialize(dictionary_name='dictionary')

Теперь добавим регуляризаторы сглаживания для всех матриц (Фи, Пси и Тета), а также регуляризатор балансирования классов для матрицы Пси:

In [9]:
model.regularizers.add(artm.SmoothSparsePhiRegularizer(name='SmoothPsiRegularizer', class_ids=[labels_class]))
model.regularizers.add(artm.LabelRegularizationPhiRegularizer(name='LabelPsiRegularizer', class_ids=[labels_class]))

model.regularizers.add(artm.SmoothSparsePhiRegularizer(name='SmoothPhiRegularizer', class_ids=[tokens_class]))
model.regularizers.add(artm.SmoothSparseThetaRegularizer(name='SmoothThetaRegularizer'))

Всё, можно приступать к обучению модели и подсчёту необходимых метрик. Во время каждой итерации прохода по коллекции будем обновлять значения коэффициентов регуляризации, весов модальностей и числа итераций прохода по документу, после чего вызывать метод обучения. В том случае, если на данной итерации нужно производить подсчёт функционалов качества, будем делать это следующим образом:
- строим матрицу Тета для тестового батча, основываясь на текущем состоянии модели;
- извлекаем матрицу Пси и вычисляем вероятности классов в документах как p(c|d) = sum_t p(c|t) * p(t|d);
- подаём этот вектор и вектор верных ответов для данного документа в функции, описанные ранее или вызванные из пакета sklearn, сохраняем результаты;
- усредняем все агрегированные данные по всем документам и печатаем результат.

In [10]:
for iter in xrange(num_collection_passes):
    print 'Start processing iteration #' + str(iter) + '...'
    model.regularizers['SmoothPsiRegularizer'].tau = smooth_psi_tau[iter]
    model.regularizers['LabelPsiRegularizer'].tau = label_psi_tau[iter]
    model.regularizers['SmoothPhiRegularizer'].tau = smooth_phi_tau[iter]
    model.regularizers['SmoothThetaRegularizer'].tau = smooth_theta_tau[iter]
    
    model.class_ids = {tokens_class: tokens_class_weight[iter], labels_class: labels_class_weight[iter]}

    model.num_document_passes = num_document_passes[iter]

    model.fit_offline(num_collection_passes=1, data_path=data_folder)
    
    test_theta = model.find_theta(data_path=data_folder, batches=[test_documents_file])
    Psi = model.get_phi(class_ids=[labels_class]).as_matrix()
    
    items_auc_roc, items_auc_pr = [], []
    one_error, is_error, precision = 0, 0, 0
    
    if iter in count_scores_iters:
        print 'Find scores for model on iter #' + str(iter) + '...'
        for item_index in xrange(len(test_theta.columns)):
            p_cd = [numpy.dot(test_theta[item_index], p_w) for p_w in Psi]

            items_auc_roc.append(sklearn.metrics.roc_auc_score(true_p_cd[item_index], p_cd))
            prec, rec, _ = sklearn.metrics.precision_recall_curve(true_p_cd[item_index], p_cd)
            items_auc_pr.append(sklearn.metrics.auc(rec, prec))

            if true_p_cd[item_index][p_cd.index(max(p_cd))] == 0:
                one_error += 1

            if not perfect_classification(true_p_cd[item_index], p_cd):
                is_error += 1

            precision += count_precision(true_p_cd[item_index], p_cd)

        average_auc       = sum(items_auc_roc) / len(items_auc_roc)
        average_auc_pr    = sum(items_auc_pr) / len(items_auc_roc)
        average_one_error = (one_error / len(items_auc_roc)) * 100
        average_is_error  = (is_error / len(items_auc_roc)) * 100
        average_precision = precision / len(items_auc_roc)

        print "AUC-ROC = %.3f " % average_auc,
        print "| OneError = %.1f " % average_one_error,
        print "| IsError = %.1f " % average_is_error,
        print "| AverPrec = %.3f " % average_precision,
        print "| AUC-PR = %.3f" % average_auc_pr

Start processing iteration #0...
Start processing iteration #1...
Start processing iteration #2...
Start processing iteration #3...
Start processing iteration #4...
Find scores for model on iter #4...
AUC-ROC = 0.964  | OneError = 65.3  | IsError = 99.8  | AverPrec = 0.117  | AUC-PR = 0.211
